# imports

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import sys
import numpy as np
import pandas as pd
import torchvision
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset
from torch import optim, cuda
import torch
from torchvision import transforms
import os
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ImageFile.LOAD_TRUNCATED_IMAGES = True
from timeit import default_timer as timer
import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
import cv2
import random
from os.path import isfile

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(2020)

# Parameters

In [3]:
learning_rate=1e-5
epochs=15
num_workers=1
batch_size=16
IMG_SIZE    = 256

# Preprocess Image

In [5]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

def load_ben_color(image, sigmaX=10 ):
    #image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    return image

# dataset and Data Loader

In [6]:
class RetinopathyDatasetTest(Dataset):
    def __init__(self, imPath, transform):
        self.imPath = imPath
        self.transform = transform
    def __len__(self):
        return 1
    
    def __getitem__(self):
        #directory='../input/aptos2019-blindness-detection/'+self.dtype+'_images'
        #img_name = os.path.join(directory, self.data.loc[idx, 'id_code'] + '.png')
        img_name=(self.imPath)
        #print(img_name)
        image = cv2.imread(img_name)
        image = load_ben_color(image, sigmaX=10 )
        image = Image.fromarray(image)
        image = self.transform(image)
        return image

In [7]:

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.55387044 ,0.5336158 , 0.5232084 ], [0.2063595 , 0.18827829, 0.15147245])
])
DetectDrImages='./images/image068.png'
test_dataset = RetinopathyDatasetTest(DetectDrImages,
                                      transform=test_transform)

## Model

In [11]:
model = torch.load("./final_traindv25.pth",map_location=torch.device('cpu'))

In [13]:
image=test_dataset.__getitem__()
image=image.unsqueeze(0)
pred=model(image.to(device))
print (pred)

tensor([[0.0994]], grad_fn=<AddmmBackward>)
